In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
import requests
import threading
import nltk
import json
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.preprocessing import normalize

In [ ]:
1 - spatial.distance.cosine([1,0,0], [0,0,0.01])

0.0

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Polzovatel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Polzovatel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('russian')
stemmer = SnowballStemmer(language='russian')

In [ ]:
headersForVacancy = {'Authorization': '#'}
def getVacancy(spec_words_lst, all_filtered_tokens, lock2, vacId):
  
  filtered_tokens = []
  try:
      response = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy)
  except:
      response = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy)
  if response.status_code != 200:
    print('error')
  vac = response.json()
  while 'errors' in vac:
        vac = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy).json()
  
  desc = vac['name'].lower() + ' '

  for k in vac['key_skills']:
      desc += k['name'].lower() + ' '
  # print(vac['key_skills'])
  soup = BeautifulSoup(vac['description'], 'html.parser')
  desc += soup.text.lower()
  token_desc1 = word_tokenize(desc, language='russian')
  # print(vac)
  # print(token_desc1)
  for token in token_desc1:
      if token not in stop_words and token not in punct and 'ə' not in token:
          stemmed_token = stemmer.stem(token)
          filtered_tokens.append(stemmed_token)
          all_filtered_tokens.add(stemmed_token) 
  # print(filtered_tokens)
  spec_words_vac = ' '.join(filtered_tokens)
  # print(spec_words_vac)
  spec_words_lst.append(spec_words_vac)
  lock2.release()

In [ ]:
def setVacancyInfo(vacancies, spec_words_lst, all_filtered_tokens):
  
  lock2 = threading.Semaphore(100)
  thread_pool2 = []
  for vacId in set([vacancy['id'] for vacancy in vacancies]):
      thread = threading.Thread(target=getVacancy, args=(spec_words_lst, all_filtered_tokens, lock2, vacId))
      thread_pool2.append(thread)
      thread.start()
      lock2.acquire()

  for thread in thread_pool2:
      thread.join()
  # lock.release()



In [ ]:
headersForVacancy = {'Authorization': '#'}

lst = [i for i in range(113, 147)]

searchParams = {'period': '30', 'per_page': '100', 'responses_count_enabled': 'false'}

punct = string.punctuation.replace("#", "") + '—' + '”' + '“' + '``' + '«' + '»' + '•' + '/'

lock = threading.Semaphore(100)
def create_global_dict(specs_id_lst):
    global_spec_words_dict = {}
    for n in specs_id_lst:
        vacancies_ids = []
        
        # для 2к вакнсий
        spec_words_dict = {}
        spec_words_lst = []
        all_filtered_tokens = set()
        
        thread_pool = []
        t = time.time()
        for i in range(10):
            print(len(spec_words_lst))
            response = requests.get(f'https://api.hh.ru/vacancies/?period={searchParams["period"]}&per_page={searchParams["per_page"]}&responses_count_enabled={searchParams["responses_count_enabled"]}&page={i}&professional_role={n}', headers=headersForVacancy)
            setVacancyInfo(response.json()['items'], spec_words_lst, all_filtered_tokens)
        
        vectorizer = CountVectorizer(vocabulary=list(all_filtered_tokens))
        X = vectorizer.fit_transform(spec_words_lst)
        transformer = TfidfTransformer()
        X_tfidf = transformer.fit_transform(X)
        X_tfidf_mean = np.mean(X_tfidf.toarray(), axis=0)

        # print(len(spec_words_lst))
        spec_words_dict = dict(zip(all_filtered_tokens, X_tfidf_mean))
        sorted_spec_words_dict = {}
        
        
        sorted_keys = sorted(spec_words_dict, key=spec_words_dict.get, reverse=True)
        
        for w in sorted_keys:
            sorted_spec_words_dict[w] = spec_words_dict[w]

        # print(sorted_spec_words_dict)

        spec_words_dict_final = {}
        for index, key in enumerate(sorted_spec_words_dict):
            if index >= 500:
                break
            else:
                spec_words_dict_final[key] = sorted_spec_words_dict[key]
        # print(len(spec_words_dict_final))
        print(time.time() - t)
        global_spec_words_dict[n] = list(spec_words_dict_final.keys())
        print(len(global_spec_words_dict))
        print(global_spec_words_dict)
        try:
            with open("roles_dictionary.json", "a") as f:
                f.write(json.dumps(global_spec_words_dict, ensure_ascii=False))
        except Exception as e:
            print(e)
        finally:
            global_spec_words_dict.clear()
        


In [ ]:
# create_global_dict(lst)

0
100
200
300
error
error
400
500
600
700
800
900
41.10969161987305
1
{113: ['работ', 'настройк', 'оп', 'администрирован', 'оборудован', 'знан', 'windows', 'сетев', 'сервер', 'поддержк', 'пк', 'обеспечен', 'сет', 'системн', 'компан', 'техническ', '00', 'пользовател', 'администратор', 'linux', 'рабоч', 'установк', 'сист', 'систем', 'обслуживан', 'server', 'ит', 'услов', 'программн', 'инфраструктур', 'дан', 'требован', 'ms', 'обязан', 'умен', 'офисн', 'офис', '1с', 'оргтехник', 'баз', 'техник', 'сервис', 'информацион', 'компьютерн', 'навык', 'команд', 'ремонт', 'ос', 'компьютер', 'пониман', 'график', 'серверн', 'мест', 'локальн', 'возможн', 'ip', 'принцип', 'мониторинг', 'работа', 'подключен', 'dns', 'корпоративн', 'плат', 'технолог', 'sql', 'заработн', 'тк', 'dhcp', 'организац', 'сотрудник', 'удален', 'решен', 'образован', 'рф', 'управлен', 'развит', 'задач', 'оформлен', 'microsoft', 'официальн', 'безопасн', 'программ', 'телефон', 'наш', 'нов', 'профессиональн', 'высш', 'уровн', 'tcp', 

100
200
error
error
error
300
400
500
600
error
error
error
700
800
900
40.590182304382324
1
{115: ['работ', 'оборудован', 'ремонт', 'обслуживан', 'техническ', '00', 'услов', 'оп', 'требован', 'обязан', 'монтаж', 'сист', 'умен', 'слесар', 'сборк', 'график', 'знан', 'образован', 'тк', 'рф', 'рабоч', 'компан', 'плат', 'работа', 'эксплуатац', 'оформлен', 'чертеж', 'заработн', 'выполнен', 'команд', 'детал', 'трудоустройств', 'средн', 'мест', 'официальн', 'узл', 'профессиональн', 'техник', 'слесарн', 'проведен', 'металлоконструкц', 'ремонтн', 'ответствен', 'специальн', 'технологическ', 'производств', 'сантехник', 'месяц', 'лет', 'прибор', 'пк', 'полн', 'инструмент', 'пользовател', 'вентиляц', 'возможн', 'кип', 'навык', 'пакет', 'здан', 'контрол', 'год', 'оплат', 'устройств', 'согласн', '30', 'стабильн', 'чтен', 'испытан', 'соответств', 'производствен', 'систем', 'своевремен', 'ремонтник', 'предприят', 'смен', 'кондиционирован', 'питан', 'документац', 'налич', 'инженерн', 'устранен', 'объект

100
200
error
300
error
400
500
600
700
800
900
error
error
39.809810400009155
1
{117: ['работ', 'кадров', 'трудов', 'веден', 'делопроизводств', 'учет', 'оформлен', 'сотрудник', '00', '1с', 'знан', 'кадр', 'отпуск', 'работник', 'персона', 'оп', 'рабоч', 'рф', 'законодательств', 'документ', 'персонал', 'компан', 'специалист', 'график', 'услов', 'увольнен', 'управлен', 'требован', 'подбор', 'тк', 'отчетн', 'обязан', 'времен', 'личн', 'организац', 'перевод', 'отдел', 'полн', 'образован', 'подготовк', 'документооборот', 'объем', 'табел', 'высш', 'книжек', 'программ', 'составлен', 'прием', 'работа', 'умен', 'плат', 'документац', 'заработн', 'дел', 'контрол', 'при', 'договор', 'пользовател', 'зуп', 'пк', 'офис', 'больничн', 'формирован', 'лет', 'штатн', 'профессиональн', 'соответств', 'ответствен', 'официальн', 'приказ', 'зарплат', 'администрирован', 'воинск', 'лист', 'трудоустройств', 'вопрос', 'хранен', 'многозадачн', 'корпоративн', 'информац', 'расписан', 'уверен', 'предприят', 'вниматель

100
200
error
300
400
500
600
700
800
error
900
40.327434062957764
1
{119: ['работ', 'тендер', 'участ', 'фз', 'подготовк', 'тендерн', 'закупк', '00', 'закупок', 'договор', 'компан', 'документац', 'оп', 'знан', 'государствен', 'электрон', 'веден', 'документ', '44', 'специалист', 'услов', 'требован', 'площадк', 'переговор', 'делов', 'торг', 'информац', 'контракт', 'продаж', 'оформлен', 'заключен', 'отдел', 'контрол', '223', 'процедур', 'умен', 'заявок', 'обязан', 'клиент', 'коммерческ', 'работа', 'поиск', 'торгов', 'заказчик', 'переписк', 'проведен', 'пользовател', 'аукцион', 'образован', 'пк', 'результат', 'конкурсн', 'грамотн', 'анализ', 'сфер', 'график', 'техническ', 'конкурс', 'рф', 'высш', 'поставщик', 'соответств', 'сопровожден', 'менеджер', 'товар', 'рабоч', 'подач', 'навык', 'необходим', 'организац', 'плат', 'услуг', 'заработн', 'команд', 'тк', 'предложен', 'полн', 'больш', 'мониторинг', 'срок', 'пакет', 'официальн', 'запрос', 'составлен', 'объем', 'офис', 'обеспечен', 'ответстве

100
200
300
400
500
600
700
error
error
errorerror

800
900
error
error
42.0975444316864
1
{121: ['работ', 'техническ', 'поддержк', 'настройк', 'пользовател', 'оборудован', 'оп', 'знан', 'компан', 'пк', '00', 'клиент', 'специалист', 'команд', 'windows', 'систем', 'сетев', 'ит', '1с', 'возможн', 'наш', 'грамотн', 'рабоч', 'удален', 'услов', 'работа', 'умен', 'решен', 'требован', 'график', 'сет', 'обязан', 'сист', 'установк', 'язык', 'обеспечен', 'обучен', 'информацион', 'офис', 'обслуживан', 'сотрудник', 'программн', 'навык', 'техник', 'корпоративн', 'образован', 'профессиональн', 'задач', 'английск', 'оформлен', 'администрирован', 'телефон', 'реч', 'продукт', 'уровн', 'нов', 'обращен', 'инженер', 'it', 'плат', 'развива', 'центр', 'заработн', 'рф', 'подключен', 'мест', 'тк', 'котор', 'ремонт', 'вопрос', 'ms', 'технолог', 'информац', 'дан', 'официальн', 'сервис', 'программ', 'развит', 'высш', 'связ', 'сво', 'компьютерн', 'баз', 'проект', 'принцип', 'стабильн', 'рост', 'компьютер', 'пробл

...

100
200
300
400
500
600
700
error
800
900
41.02582263946533
1
{137: ['финансов', 'работ', 'отчетн', 'компан', 'анализ', 'контрол', 'управленческ', 'учет', 'оп', 'знан', 'and', 'проект', 'планирован', '1с', 'управлен', 'бюджетирован', 'подготовк', '00', 'экономическ', 'бухгалтерск', 'налогов', 'бизнес', 'формирован', 'банк', 'услов', 'бюджет', 'мсфо', 'организац', 'деятельн', 'предприят', 'требован', 'of', 'директор', 'веден', 'участ', 'обязан', 'образован', 'процесс', 'менеджер', 'умен', 'финанс', 'высш', 'документ', 'excel', 'клиент', 'команд', 'разработк', 'отчет', 'лет', 'работа', 'средств', 'инвестицион', 'план', 'расчет', 'the', 'ms', 'информац', 'составлен', 'плат', 'та', 'показател', 'заработн', 'дан', 'денежн', 'график', 'in', 'оформлен', 'возможн', 'систем', 'офис', 'навык', 'результат', 'договор', 'отдел', 'бухгалтер', 'пользовател', 'кредитн', 'рф', 'автоматизац', 'корпоративн', 'аналитическ', 'эффективн', 'больш', 'развит', 'тк', 'задач', 'наш', 'профессиональн', 'соответст

100
200
300
400
500
600
700
740
740
30.3906991481781
1
{139: ['работ', 'фотограф', 'фотосъемк', 'съемк', 'компан', 'adobe', 'команд', '00', 'наш', 'услов', 'photoshop', 'требован', 'смартфон', 'маршрут', 'умен', 'фот', 'клиент', 'оп', 'работа', 'обработк', 'график', 'возможн', 'продаж', 'обязан', 'сво', 'врем', 'навык', 'грамотн', 'предметн', 'ваш', 'пк', 'интернет', 'реч', 'обучен', 'портретн', 'ответствен', 'ретуш', 'пользовател', 'город', 'качеств', 'товар', 'развива', 'знан', 'профессиональн', 'студ', 'внимательн', 'котор', 'информац', 'рабоч', 'выполнен', 'проект', 'дан', 'оплат', 'желан', 'lightroom', 'люд', 'фотостуд', 'налич', 'эт', 'детск', 'месяц', 'активн', 'дет', 'дела', 'оборудован', 'ищет', 'студийн', 'помога', 'опыт', '10', 'программ', 'сотрудник', 'так', 'обеспечен', 'получен', 'репортажн', 'кажд', 'поездк', 'программн', 'суток', 'исполнител', 'нам', 'продукц', 'сфер', 'документ', 'техник', 'оформлен', 'коммуникабельн', 'предлага', 'пунктуальн', 'услуг', 'рост', 'изобра

100
200
300
400
500
error
error
600
700
800
900
40.3979549407959
1
{141: ['работ', 'мебел', '00', 'производств', 'продаж', 'одежд', 'услов', 'оп', 'издел', 'клиент', 'компан', 'требован', 'обязан', 'команд', 'работа', 'знан', 'график', 'умен', 'шве', 'дизайнер', 'оформлен', 'дизайн', 'ответствен', 'заказ', 'пош', 'плат', 'наш', 'швейн', 'портн', 'желан', 'рабоч', 'обучен', 'заработн', 'возможн', 'программ', 'технолог', 'тк', 'проект', 'навык', 'аккуратн', 'оборудован', 'оплат', 'грамотн', 'пк', 'пошив', 'рф', 'официальн', 'высок', 'нов', 'полн', 'материал', 'разработк', 'салон', 'месяц', 'пользовател', 'треб', 'год', 'стабильн', 'профессиональн', 'лет', 'продукц', '000', 'трудоустройств', 'образован', 'контрол', 'реч', 'качеств', 'сдельн', 'менеджер', 'мест', 'дружн', 'внимательн', 'ремонт', 'конструктор', 'коллект', 'сотрудник', 'кухон', 'развива', 'корпусн', 'модел', 'труд', '10', 'выполнен', 'цех', 'ткан', 'готов', 'офис', 'веден', 'сво', 'индивидуальн', 'оклад', 'активн', 'техничес

100
200
300
400
500
600
700
800
900
40.48564386367798
1
{143: ['работ', 'ремонт', 'электрооборудован', 'обслуживан', 'монтаж', 'оборудован', 'услов', '00', 'требован', 'электромонтер', 'оп', 'электрик', 'электрическ', 'техническ', 'обязан', 'сет', 'групп', 'электромонтажник', 'умен', 'знан', 'график', 'образован', 'электробезопасн', 'схем', 'устройств', 'выполнен', 'команд', 'объект', 'плат', 'официальн', 'рф', 'тк', 'трудоустройств', 'компан', 'заработн', 'эксплуатац', 'оформлен', 'электромонтажн', 'работа', 'профессиональн', 'рабоч', 'лет', 'вахтов', 'специальн', 'сист', 'чтен', 'сборк', 'согласн', 'ответствен', '30', 'кабельн', 'пк', 'разряд', 'средн', 'полн', 'электротехническ', 'метод', 'кабел', 'налич', 'мест', 'силов', 'установк', 'допуск', 'производств', 'своевремен', 'пользовател', 'возможн', 'обеспечен', 'питан', 'месяц', 'лин', 'ниж', 'оплат', 'соответств', 'проживан', 'оплачива', 'прокладк', 'кв', 'проведен', 'пакет', 'документац', 'мен', 'распределительн', 'состоян', 'сист

100
200
300
400
500
error
error
600
700
800
900
39.14311909675598
1
{145: ['работ', 'договор', 'юридическ', 'компан', 'правов', 'прав', 'документ', '00', 'оп', 'подготовк', 'суд', 'юрист', 'знан', 'сопровожден', 'договорн', 'законодательств', 'судебн', 'составлен', 'веден', 'услов', 'корпоративн', 'рф', 'деятельн', 'гражданск', 'орган', 'требован', 'клиент', 'грамотн', 'организац', 'обязан', 'умен', 'вопрос', 'делов', 'оформлен', 'заключен', 'арбитражн', 'образован', 'государствен', 'юрисконсульт', 'высш', 'интерес', 'банкротств', 'лиц', 'работа', 'претензион', 'проект', 'реч', 'рабоч', 'трудов', 'офис', 'график', 'исков', 'участ', 'разработк', 'претенз', 'согласован', 'недвижим', 'лет', 'дел', 'тк', 'процессуальн', 'возможн', 'переписк', 'представлен', 'плат', 'заработн', 'общ', 'отдел', 'ответствен', 'предприят', 'взаимодейств', 'профессиональн', 'контрагент', 'пользовател', 'пк', 'команд', 'помощник', 'полн', 'информац', 'анализ', 'экспертиз', 'заявлен', 'сфер', 'обществ', '18', 'ис

In [ ]:
headersForResume = {'Authorization': '#'}

def getResume(id):
    resumeResponse = requests.get('https://api.hh.ru/resumes/' + id, headers=headersForResume)
    if resumeResponse.status_code == 200:
        return resumeResponse.json()
    else:
        return False

In [ ]:
resume = getResume('09c93be900004f2d620039ed1f736563726574')
role_id1 = [role['id'] for role in resume['professional_roles']]
print(role_id1)
print(resume)

['96']
{'last_name': None, 'first_name': None, 'middle_name': None, 'title': 'программист C#', 'created_at': '2008-12-16T09:36:17+0300', 'updated_at': '2021-12-06T16:26:44+0300', 'area': {'id': '4', 'name': 'Новосибирск', 'url': 'https://api.hh.ru/areas/4'}, 'age': 36, 'gender': {'id': 'male', 'name': 'Мужской'}, 'salary': {'amount': 300000, 'currency': 'RUR'}, 'photo': None, 'total_experience': {'months': 150}, 'certificate': [], 'hidden_fields': [], 'actions': {'download': {'pdf': {'url': 'https://api.hh.ru/resumes/09c93be900004f2d620039ed1f736563726574/download/%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20C%23.pdf?type=pdf'}, 'rtf': {'url': 'https://api.hh.ru/resumes/09c93be900004f2d620039ed1f736563726574/download/%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20C%23.rtf?type=rtf'}}}, 'alternate_url': 'https://hh.ru/resume/09c93be900004f2d620039ed1f736563726574', 'id': '09c93be900004f2d620039ed1f736563726574', 'download': {'pdf': {'url': '

In [ ]:
#09c93be900004f2d620039ed1f736563726574
def get_resume_vector(id):
    filtered_tokens = []
    resume_words_lst = []
    
    resume = getResume(id)
    role_id1 = [role['id'] for role in resume['professional_roles']]
    string = resume['title'].lower() + ' '
    for word in resume['skill_set']:
        string += word.lower() + ' '
    for dct in resume['experience']:
        string += dct['description'].lower() + ' '
    if resume['skills']:
        string += resume['skills'].lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              filtered_tokens.append(stemmed_token)
    resume_words = ' '.join(filtered_tokens)
    with open("roles_dictionary.json", "r") as f:
        dictionary = json.loads(f.read())
    resume_words_lst.append(resume_words)
    vectorizer = CountVectorizer(vocabulary=dictionary[role_id1[0]]) # здесь возможно нужно будет сделать по разным ролям, если роль не одна
    vector_resume = vectorizer.fit_transform(resume_words_lst)
    return vector_resume.toarray()

In [ ]:
get_resume_vector('09c93be900004f2d620039ed1f736563726574')

array([[ 0,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  2,  7,  1,  0,
         0,  7,  1,  0, 10,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  1,  0,  1,  0,  0,  2,  0,  8,  3,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  1,  0,  0,  3,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  6,
         0,  0,  0,  2,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  2,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  3,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
def get_a_vacancy(id):
    vacancyResponse = requests.get('https://api.hh.ru/vacancies/' + id, headers=headersForVacancy)
    if vacancyResponse.status_code == 200:
        return vacancyResponse.json()
    else:
        return False

In [ ]:
get_a_vacancy('42222926')

{'id': '42222926',
 'premium': True,
 'billing_type': {'id': 'premium', 'name': 'Премиум'},
 'relations': [],
 'name': 'Разработчик C#',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '3',
  'name': 'Екатеринбург',
  'url': 'https://api.hh.ru/areas/3'},
 'salary': {'from': 120000, 'to': None, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Екатеринбург',
  'street': 'улица Шейнкмана',
  'building': '55',
  'description': None,
  'lat': 56.832034,
  'lng': 60.587539,
  'raw': 'Екатеринбург, улица Шейнкмана, 55',
  'metro': None,
  'metro_stations': []},
 'allow_messages': True,
 'site': {'id': 'hh', 'name': 'hh.ru'},
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'schedule': {'id': 'remote', 'name': 'Удаленная работа'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': {'name': 'Харченко Мария',
  'email': 'hr@knopka.com',
  'phones': [{'co

In [ ]:
def get_vacancy_vector(id):
    filtered_tokens = []
    vacancy_words_lst = []
    
    vacancy = get_a_vacancy(id)
    role_id1 = [role['id'] for role in vacancy['professional_roles']]
    string = vacancy['name'].lower() + ' '
    for word in vacancy['key_skills']:
        string += word['name'].lower() + ' '
    soup = BeautifulSoup(vacancy['description'], 'html.parser')
    string += soup.text.lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              filtered_tokens.append(stemmed_token)
    vacancy_words = ' '.join(filtered_tokens)
    with open("roles_dictionary.json", "r") as f:
        dictionary = json.loads(f.read())
    vacancy_words_lst.append(vacancy_words)
    vectorizer = CountVectorizer(vocabulary=dictionary[role_id1[0]])
    vector_vacancy = vectorizer.fit_transform(vacancy_words_lst)
    return vector_vacancy.toarray()

In [ ]:
get_vacancy_vector('42222926')

array([[1, 9, 1, 2, 0, 5, 1, 5, 3, 5, 4, 1, 7, 2, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 4, 2, 1, 0, 7, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 6, 1, 2,
        0, 1, 3, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 1, 0,
        0, 2, 1, 0, 4, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

In [ ]:
def get_cosine_similarity(vec1, vec2):
    similarity = 1 - spatial.distance.cosine(vec1, vec2)
    if similarity > 0.5:
        return 0.5
    return 1 - spatial.distance.cosine(vec1, vec2)

In [ ]:
get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('09c93be900004f2d620039ed1f736563726574'))

0.14222673479551629

In [ ]:
get_cosine_similarity(get_vacancy_vector('49869318'), get_resume_vector('09c93be900004f2d620039ed1f736563726574'))

0.14861295035835198

In [ ]:
get_cosine_similarity(get_vacancy_vector('49869318'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.2845621368982486

In [ ]:
get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.23108538743337892

In [ ]:
get_cosine_similarity(get_vacancy_vector('48952810'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.3019104835450963

In [ ]:
def get_key_skills_evaluation(res_id, vac_id):
    res = getResume(res_id)
    vac = get_a_vacancy(vac_id)
    res_lst = []
    vac_lst = []
    counter = 0
    
    for word in res['skill_set']:
        res_lst.append(word.lower())
    for word in vac['key_skills']:
        vac_lst.append(word['name'].lower())
    for i in res_lst:
        if i in vac_lst:
            counter += 1
    print(res_lst)
    print(vac_lst)
    if len(vac_lst) > 0:
        evaluation = (1/len(vac_lst)*counter)/2
    else:
        evaluation = 0
    if evaluation > 0.5:
        return 0.5
    return evaluation

In [ ]:
get_key_skills_evaluation('1a6443020008943f530039ed1f64483378476e', '48952810')

['ответственность', 'коммуникабельность', 'внимательность', 'трудолюбие', 'желание учиться', 'честность', 'грамотность', 'работа в команде', 'fiddler', 'api', 'test case', 'пользователь пк', 'html', 'css', 'qa', 'scrum', 'atlassian jira', 'тестирование', 'postman', 'json api', 'регресcионное тестирование', 'функциональное тестирование']
[]


0

In [ ]:
def get_evaluation(similarity, evaluation):
    return similarity + evaluation

In [ ]:
get_evaluation(get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('09c93be900004f2d620039ed1f736563726574')), get_key_skills_evaluation('09c93be900004f2d620039ed1f736563726574', '42222926'))

['ado.net', 'ajax', 'asn1', 'c#', 'certificate', 'delphi', 'dewexpress', 'entityframework', 'git', 'linq', 'ms silverlight', 'ms sql', 'my sql', 'net framework 2-4', 'oracle', 'php', 'spring.net', 'svn', 'telerik', 'tfs', 't-sql', 'vb', 'wcf', 'windows installer xml', 'wpf', 'xml', 'xpath', 'xsd', 'nhibernate', 'dapper']
['c#', '.net framework', 'sql', 'java']


0.2672267347955163

In [ ]:
def get_description_rate(res_id, vac_id):
    resume = getResume(res_id)
    vacancy = get_a_vacancy(vac_id)
    res_filtered_tokens = []
    vac_filtered_tokens = []
    counter = 0
    
    string = ''
    for dct in resume['experience']:
        string += dct['description'].lower() + ' '
    if resume['skills']:
        string += resume['skills'].lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              res_filtered_tokens.append(stemmed_token)

    soup = BeautifulSoup(vacancy['description'], 'html.parser')
    string = soup.text.lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              vac_filtered_tokens.append(stemmed_token)
    
    for i in res_filtered_tokens:
        if i in vac_filtered_tokens:
            counter += 1
    print(res_filtered_tokens)
    print(vac_filtered_tokens)
    if len(vac_filtered_tokens) > 0:
        rate = (1/len(res_filtered_tokens)*counter)/2
    else:
        rate = 0
    if rate > 0.5:
        return 0.5
    return rate


In [ ]:
get_description_rate('1a6443020008943f530039ed1f64483378476e', '48952810')

['рамк', 'обучен', 'ментор', 'тестирован', 'мно', 'изуч', 'след', 'основн', 'понят', 'тестирован', 'программн', 'обеспечен', 'уровн', 'тестирован', 'вид', 'тестирован', 'техник', 'тест', 'дизайн', 'класс', 'эквивалентн', 'анализ', 'граничн', 'значен', 'таблиц', 'принят', 'решен', 'попарн', 'тестирован', 'т.д.', 'регрессион', 'тестирован', 'кроссбраузерн', 'тестирован', 'ручн', 'функциональн', 'тестирован', 'сайт', 'приложен', 'мобильн', 'приложен', 'апи-сервис', 'тестирован', 'метод', 'черн', 'сер', 'ящик', 'методолог', 'тестирования\\разработк', 'пониман', 'клиент-серверн', 'архитектур', 'составлен', 'тест-план', 'чек-лист', 'тест', 'кейс', 'баг', 'репорт', 'работ', 'postman', 'soapui', 'оформлен', 'баг', 'репорт', 'jira', 'работ', 'панел', 'разработчик', 'devtools', 'базов', 'знан', 'бд', 'написан', 'прост', 'запрос', 'sql', 'select', 'join', 'перехватыван', 'мобильного\\веб', 'трафик', 'fiddler', 'использован', 'мобильн', 'ферм', 'пониман', 'протокол', 'метод', 'http', 'запросов/отв

0.10789473684210527

In [ ]:
get_description_rate('1a6443020008943f530039ed1f64483378476e', '42222926')

['рамк', 'обучен', 'ментор', 'тестирован', 'мно', 'изуч', 'след', 'основн', 'понят', 'тестирован', 'программн', 'обеспечен', 'уровн', 'тестирован', 'вид', 'тестирован', 'техник', 'тест', 'дизайн', 'класс', 'эквивалентн', 'анализ', 'граничн', 'значен', 'таблиц', 'принят', 'решен', 'попарн', 'тестирован', 'т.д.', 'регрессион', 'тестирован', 'кроссбраузерн', 'тестирован', 'ручн', 'функциональн', 'тестирован', 'сайт', 'приложен', 'мобильн', 'приложен', 'апи-сервис', 'тестирован', 'метод', 'черн', 'сер', 'ящик', 'методолог', 'тестирования\\разработк', 'пониман', 'клиент-серверн', 'архитектур', 'составлен', 'тест-план', 'чек-лист', 'тест', 'кейс', 'баг', 'репорт', 'работ', 'postman', 'soapui', 'оформлен', 'баг', 'репорт', 'jira', 'работ', 'панел', 'разработчик', 'devtools', 'базов', 'знан', 'бд', 'написан', 'прост', 'запрос', 'sql', 'select', 'join', 'перехватыван', 'мобильного\\веб', 'трафик', 'fiddler', 'использован', 'мобильн', 'ферм', 'пониман', 'протокол', 'метод', 'http', 'запросов/отв

0.09210526315789473

In [ ]:
get_description_rate('1a6443020008943f530039ed1f64483378476e', '49869318')

['рамк', 'обучен', 'ментор', 'тестирован', 'мно', 'изуч', 'след', 'основн', 'понят', 'тестирован', 'программн', 'обеспечен', 'уровн', 'тестирован', 'вид', 'тестирован', 'техник', 'тест', 'дизайн', 'класс', 'эквивалентн', 'анализ', 'граничн', 'значен', 'таблиц', 'принят', 'решен', 'попарн', 'тестирован', 'т.д.', 'регрессион', 'тестирован', 'кроссбраузерн', 'тестирован', 'ручн', 'функциональн', 'тестирован', 'сайт', 'приложен', 'мобильн', 'приложен', 'апи-сервис', 'тестирован', 'метод', 'черн', 'сер', 'ящик', 'методолог', 'тестирования\\разработк', 'пониман', 'клиент-серверн', 'архитектур', 'составлен', 'тест-план', 'чек-лист', 'тест', 'кейс', 'баг', 'репорт', 'работ', 'postman', 'soapui', 'оформлен', 'баг', 'репорт', 'jira', 'работ', 'панел', 'разработчик', 'devtools', 'базов', 'знан', 'бд', 'написан', 'прост', 'запрос', 'sql', 'select', 'join', 'перехватыван', 'мобильного\\веб', 'трафик', 'fiddler', 'использован', 'мобильн', 'ферм', 'пониман', 'протокол', 'метод', 'http', 'запросов/отв

0.05526315789473684

In [ ]:
get_description_rate('09c93be900004f2d620039ed1f736563726574', '49869318')

['moedelo.org', 'бюр', '–', 'оценк', 'контрагент', 'c', '#', 'net', 'core', 'ms', 'sql', 'postgresql', 'react', 'python', 'c', '#', 'postgresql', 'складск', 'остатк', 'зарплат', 'кадр', 'учет', 'отчетн', 'е-сайт', 'сайт', 'издан', 'холдинг', 'aspnet', 'mvc', 'ms', 'sql', 'redis', 'net', 'core', 'docker', 'unity', 'разработк', 'дантист', 'фриланс', 'сша', 'c', '#', 'wpf', 'nhibernate', '1.', 'систем', 'smss', 'c', '#', 'mssql', 'spring.net', 'высоконагружен', 'сервер', 'различн', 'систем', 'подсистем', '2.', 'систем', 'dwh', 'c', '#', 'mssql', 'spring.net', 'wpf', 'wcf', 'dewexpress', 'asn1', 'сложн', 'систем', 'сервис', 'регистрац', 'хранен', 'обработк', 'анализ', 'распределен', 'дан', 'траффик', 'голосов', 'смс', 'hlr', 'gprs', '…', 'умеет', 'работа', 'различн', 'вендор', 'alcatel', 'siemens', 'huawei', 'nokia', 'ericsson', '…', 'мног', 'верс', 'оборудован', 'вендор', 'настраива', 'различн', 'нужд', 'напр', 'работа', 'разн', 'стран', 'снг', '3.', 'систем', 'theseus', '–', 'c', '#', 'm

0.04056795131845842

In [ ]:
get_description_rate('09c93be900004f2d620039ed1f736563726574', '42222926')

['moedelo.org', 'бюр', '–', 'оценк', 'контрагент', 'c', '#', 'net', 'core', 'ms', 'sql', 'postgresql', 'react', 'python', 'c', '#', 'postgresql', 'складск', 'остатк', 'зарплат', 'кадр', 'учет', 'отчетн', 'е-сайт', 'сайт', 'издан', 'холдинг', 'aspnet', 'mvc', 'ms', 'sql', 'redis', 'net', 'core', 'docker', 'unity', 'разработк', 'дантист', 'фриланс', 'сша', 'c', '#', 'wpf', 'nhibernate', '1.', 'систем', 'smss', 'c', '#', 'mssql', 'spring.net', 'высоконагружен', 'сервер', 'различн', 'систем', 'подсистем', '2.', 'систем', 'dwh', 'c', '#', 'mssql', 'spring.net', 'wpf', 'wcf', 'dewexpress', 'asn1', 'сложн', 'систем', 'сервис', 'регистрац', 'хранен', 'обработк', 'анализ', 'распределен', 'дан', 'траффик', 'голосов', 'смс', 'hlr', 'gprs', '…', 'умеет', 'работа', 'различн', 'вендор', 'alcatel', 'siemens', 'huawei', 'nokia', 'ericsson', '…', 'мног', 'верс', 'оборудован', 'вендор', 'настраива', 'различн', 'нужд', 'напр', 'работа', 'разн', 'стран', 'снг', '3.', 'систем', 'theseus', '–', 'c', '#', 'm

0.08823529411764705